## Step 1 : Web Scraping 

In [1]:
#Import dependencies 
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
# import os
import pandas as pd
from random import randint
from time import sleep 

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\eenab\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache




In [3]:
#Create empty dictionary to hold all variables 
mars_data = {}

# title : string
# paragraph : string
# image :string
# pd.to_html -- run on the table they provide : string
# hemispheres: list of dictionaries


## NASA Mars News

In [4]:
# Visit Nasa news url through splinter module
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
sleep(randint(1,10))

In [5]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')

In [6]:
#Find latest news title 
news_title = soup.find_all('div', class_='content_title')
news_title[1].get_text()
title = news_title[1].get_text()
mars_data["title"] = title

In [7]:
#Find latest news description
news_p = soup.find_all('div', class_='article_teaser_body')
news_p[1].get_text()
mars_data["news_p"] = news_p
news_p

[<div class="article_teaser_body">A 360-degree panorama taken by the rover’s Mastcam-Z instrument will be discussed during a public video chat this Thursday.</div>,
 <div class="article_teaser_body">When the Perseverance rover safely touched down on the Martian surface, inside Jezero Crater, on Feb. 18, 2021, it was also a safe landing for the nearly 11 million names on board.</div>,
 <div class="article_teaser_body">The agency’s newest rover captured first-of-its kind footage of its Feb. 18 touchdown and has recorded audio of Martian wind.
 
 </div>,
 <div class="article_teaser_body">First-of-its kind footage from the agency’s newest rover will be presented during a briefing this morning.</div>,
 <div class="article_teaser_body">The technology demonstration has phoned home from where it is attached to the belly of NASA’s Perseverance rover. </div>,
 <div class="article_teaser_body">The six-wheeled robot’s latest data since touching down yesterday include a hi-res image captured as the

## Featured Image

In [ ]:
# Visit JPL Mars Space Images
jpl_img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_img_url)
sleep(randint(1,10))


In [ ]:
# Sort for latest Mars photos in gallery 
browser.find_by_id('filter_Mars').first.click()
sleep(randint(1,10))

In [ ]:
#Set up new html/bs
featured_img_html = browser.html
img_soup = BeautifulSoup(featured_img_html, 'html')
featured_img_url = img_soup.find('img', class_="BaseImage object-contain")["src"]
print(featured_img_url)
mars_data["image"] = featured_img_url

## Mars Facts

In [ ]:
# Visit Mars facts url through splinter module
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
sleep(randint(1,10))

In [ ]:
# Scrape the table containing facts about the planet including Diameter, Mass, etc.and turn into HTML table string
facts_table = pd.read_html(facts_url)
mars_data["facts_table"] = facts_table

## Mars Hemispheres

In [ ]:
#Set an empty list to hold urls
hemis_img_urls = []
    # {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    # {"title": "Cerberus Hemisphere", "img_url": "..."},
    # {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    # {"title": "Syrtis Major Hemisphere", "img_url": "..."},


In [ ]:
#Visit main site
hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_url)
sleep(randint(1,10))

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#Locate each of the divs with hemisphere info
base_url = "https://astrogeology.usgs.gov"

results = soup.find_all("div", class_='description')
results

In [ ]:
#Below finds one link, used as an example
results[0].find('a')['href']

In [ ]:
#Runs through each hemisphere div, clicks in to find link to full size image URL, title of Hemisphere, and appends the dictionary
for result in results:
    
    hemi_title = result.find('h3').text

    img_page_url = base_url + result.find('a')['href']
    
    response = requests.get(img_page_url)
    img_page_soup = BeautifulSoup(response.text, 'html.parser')
    
    hemi_img_url = img_page_soup.find('ul').li.a['href']
    
    hemi_img_dict = {'title': hemi_title, 
                'img_url': hemi_img_url}
    
    hemis_img_urls.append(hemi_img_dict)


In [ ]:
hemis_img_urls

In [ ]:
mars_data

In [ ]:
browser.quit()